In [19]:
import sys
sys.path.append("./expectation_value")

from expectationvalue import ExpVal
import numpy as np
import qiskit.quantum_info as qi

In [20]:
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
import qiskit.quantum_info as qif


class MoleculeHamiltonian():
    def __init__(self,
                 molecule,
                 converter,
                 basis,
                ):
        self.molecule = molecule
        self.basis = basis
        driver = PySCFDriver(
            atom= self.molecule,
            basis=self.basis,
            charge=0,
            spin=0,
            unit=DistanceUnit.ANGSTROM,
            )
        
        self.problem = driver.run()
        self.converter = converter
        
    def Hamiltonian(self,
                    freeze_core = True,
                    remove_orbitals = None
                   ):
        problem = self.problem
        fermionic_op = self.problem.hamiltonian.second_q_op()
            
        if freeze_core:
            fc_transformer = FreezeCoreTransformer(freeze_core=freeze_core, remove_orbitals=remove_orbitals)
            problem = fc_transformer.transform(problem)
            fermionic_op = problem.hamiltonian.second_q_op()
            
        qubit_op = self.converter.convert(fermionic_op,
                                           sector_locator=problem.symmetry_sector_locator)
        return qubit_op
    
    def ComputeGroundState(self , circuit = True):
        solver = NumPyMinimumEigensolver()
        
        ground_solver = GroundStateEigensolver(self.converter, solver)
        ground_state = ground_solver.solve(self.problem)
        
        state = ground_state.groundstate[0]
        eigenvalue = ground_state.groundenergy
        
        if circuit:
            return state, eigenvalue
        else:
            return qif.Statevector(state).data, eigenvalue

In [21]:
dist = 1.0
molecules = {
    "H2Be": "Be .0 .0 .0; H .0 .0 -" + str(dist) + "; H .0 .0 " + str(dist),
    "H2":"H .0 .0 .0; H .0 .0 " + str(dist),
    "LiH":"Li .0 .0 .0; H .0 .0 " + str(dist),
    "H2O": "H -0.0399 -0.0038 0.0; O 1.5780 0.8540 0.0; H 2.7909 -0.5159 0.0",
    "NH4": "N 0.0 0.0 0.149; H 0.0 0.947 -0.348; H  0.821 -0.474 -0.348; H -0.821 -0.474 -0.348"
}

In [22]:
mol = 'H2'
# mol = 'H2Be'
# mol = 'LiH'
# mol = "H2O"
# mol = "NH4"

In [23]:
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter


converter = QubitConverter( JordanWignerMapper(), z2symmetry_reduction=None)
molecular_hamiltonian = MoleculeHamiltonian(molecules[mol] , converter=converter, basis='sto3g')
hamiltonian = molecular_hamiltonian.Hamiltonian( freeze_core = None )
problem = molecular_hamiltonian.problem
n_qubits = hamiltonian.num_qubits

In [24]:
def get_number_nbody_terms(qubit_op):
    
    n_bodies_inter = []
    
    for i in range(len(qubit_op)):
        pauli_string = qubit_op[i].to_pauli_op().primitive.to_label()
        num_x = pauli_string.count('X')
        num_y = pauli_string.count('Y')
        
        n_body_terms = num_x+num_y
        
        n_bodies_inter.append(n_body_terms)
        
    return list(set(n_bodies_inter))

bodies = get_number_nbody_terms(hamiltonian)

print(bodies)

[0, 4]


In [25]:
circ_gs, eig_gs = molecular_hamiltonian.ComputeGroundState()

In [26]:
algorithm = ExpVal(n_shots = 10000,
        bodies = bodies, 
        r = 4, 
        r_shots = 1000,
        n_qubits = n_qubits)

In [ ]:
shadows = algorithm.get_intefereces()